In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np 
import open3d as o3d
import os.path as osp
import os
import time

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
pc = o3d.io.read_point_cloud(f'/media/kluo/Stephany/v2x_dataset/MergedPointCloud/mini_7/merged_50.pcd')
points = np.asarray(pc.points)
colors = np.asarray(pc.colors)

In [7]:
colors

array([[0.8, 0.4, 0.6],
       [0.8, 0.4, 0.6],
       [0.8, 0.4, 0.6],
       ...,
       [1. , 0.6, 0.2],
       [1. , 0.6, 0.2],
       [1. , 0.6, 0.2]])

In [16]:
# view_status = """{
# 	"class_name" : "ViewTrajectory",
# 	"interval" : 29,
# 	"is_loop" : false,
# 	"trajectory" : 
# 	[
# 		{
# 			"boundingbox_max" : [ 218.3385009765625, 76.248146057128906, 20.960601806640625 ],
# 			"boundingbox_min" : [ -180.83258056640625, -149.74227905273438, -11.554059028625488 ],
# 			"field_of_view" : 60.0,
# 			"front" : [ 0.0037887345655083175, -0.026423238908849973, 0.99964366548083428 ],
# 			"lookat" : [ -28.386351918275182, -61.092354458115807, 1.0199136050344466 ],
# 			"up" : [ -0.18663836077341517, 0.98206672189830169, 0.026666008845301955 ],
# 			"zoom" : 0.13999999999999962
# 		}
# 	],
# 	"version_major" : 1,
# 	"version_minor" : 0
# }"""

view_status = """{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 218.3385009765625, 76.248146057128906, 20.960601806640625 ],
			"boundingbox_min" : [ -180.83258056640625, -149.74227905273438, -11.554059028625488 ],
			"field_of_view" : 60.0,
			"front" : [ 0.14969908237785134, -0.80852446648900111, 0.56910312933940677 ],
			"lookat" : [ -33.459614649965793, -65.170023496217837, -3.7534205763743964 ],
			"up" : [ -0.072208736619961217, 0.56511189561377617, 0.82184818780085778 ],
			"zoom" : 0.059999999999999609
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}"""

In [17]:
# scene_idx = 7
max_frames = 251

vis_range_start = 50
vis_range_end = 250
# save_dir = f"./vis_{scene_idx}/"
# os.makedirs(save_dir, exist_ok=True)

In [21]:
for scene_idx in range(28, 38):
    save_dir = f"./vis_{scene_idx}/"
    os.makedirs(save_dir, exist_ok=True)
    
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    vis.set_full_screen(True)
    vis.get_render_option().background_color = [0.01, 0, 0]
    vis.get_render_option().point_size = 1.5

    # create pcd object
    point_cloud_o3d = o3d.geometry.PointCloud()  # Important to use same object for repeated call to update_geometry

    start_time = time.time()
    prev_sec = -1
    idx = 45
    first_call = True

    while idx < max_frames:
        curr_sec = int(time.time() - start_time)
        if curr_sec - prev_sec >= 0.01:  # Check if 1 second has passed
            prev_sec = curr_sec

            # Load pointcloud
            pc = o3d.io.read_point_cloud(f'/media/kluo/Stephany/v2x_dataset/MergedPointCloud/mini_{scene_idx}/merged_{idx}.pcd')
            points = np.asarray(pc.points)
            colors = np.asarray(pc.colors)

            # Modify original pointcloud a little to show a video
            # Note - Always edit in-place.
            point_cloud_o3d.points = o3d.utility.Vector3dVector(points)
            point_cloud_o3d.colors = o3d.utility.Vector3dVector(colors)

            # add or update geometry objects
            if first_call:
                vis.add_geometry(point_cloud_o3d)  # add point cloud
                vis.set_view_status(view_status)
                first_call = False  # change flag
            else:
                vis.update_geometry(point_cloud_o3d)  # update point cloud

            if idx >= vis_range_start and idx < vis_range_end:
                vis.capture_screen_image(osp.join(save_dir, "%03d.jpg" % idx))

            idx += 1

        # update o3d window
        if not vis.poll_events():
            break
        vis.update_renderer()

    vis.close()
    vis.destroy_window()